In [235]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from keras.models import Sequential
from keras.layers import Dense,RepeatVector,Dropout
from keras.layers import Flatten
#from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.optimizers import Adam
import time

In [236]:
data = pd.read_csv('PEP.csv')


In [237]:
data

,ticker,date,open,high,low,close
0,PEP,1972-06-01,0.000,1.60,1.580,1.60
1,PEP,1972-06-02,1.600,1.62,1.600,1.60
2,PEP,1972-06-05,1.600,1.60,1.540,1.57
3,PEP,1972-06-06,1.560,1.56,1.560,1.56
4,PEP,1972-06-07,1.560,1.56,1.550,1.55
...,...,...,...,...,...,...
12826,PEP,2023-04-13,182.620,184.61,182.105,184.38
12827,PEP,2023-04-14,183.335,184.02,182.920,183.51
12828,PEP,2023-04-17,184.190,184.75,183.481,184.45
12829,PEP,2023-04-18,184.770,185.22,183.740,184.83


In [238]:
data = data.filter(['close'])


In [239]:
# Lấy 100 dòng cuối cùng của df
data = data.tail(1000)
data

,close
11831,127.24
11832,126.83
11833,127.67
11834,127.27
11835,125.98
...,...
12826,184.38
12827,183.51
12828,184.45
12829,184.83


In [240]:
#Tiền xử lý
data.fillna(0, inplace=True)

# Normalize data
scaler = MinMaxScaler()
dataScale = scaler.fit_transform(data.values.reshape(-1,1))

In [241]:
dataScale

array([[0.28437233],
       [0.2793705 ],
       [0.28961815],
       [0.28473832],
       [0.26900085],
       [0.26948884],
       [0.26985482],
       [0.29376601],
       [0.28339636],
       [0.28644626],
       [0.31011346],
       [0.32133707],
       [0.32426497],
       [0.31792119],
       [0.30645358],
       [0.31889716],
       [0.3176772 ],
       [0.31194339],
       [0.28913017],
       [0.29571795],
       [0.30108576],
       [0.29364402],
       [0.30559961],
       [0.30669757],
       [0.33500061],
       [0.34219837],
       [0.36037575],
       [0.3618397 ],
       [0.35708186],
       [0.35634988],
       [0.35390997],
       [0.35134805],
       [0.34878614],
       [0.34317433],
       [0.352812  ],
       [0.36879346],
       [0.36635354],
       [0.36854947],
       [0.36452361],
       [0.34634622],
       [0.33866049],
       [0.33182872],
       [0.34085641],
       [0.34768818],
       [0.36671953],
       [0.35488593],
       [0.34927412],
       [0.339

In [242]:
#Chia dữ liệu thành train test
data = pd.DataFrame(dataScale)

#dataScale = data
train_percent = 0.7
train_size = int(len(data) * train_percent)
train_data = data.iloc[:train_size]
test_data = data.iloc[train_size:]

In [243]:
data = train_data

#dataScale = data
train_percent = 0.7
train_size = int(len(data) * train_percent)
train_data = data.iloc[:train_size]
test_data = data.iloc[train_size:]

In [244]:
#Chuẩn bị dữ liệu để đưa vào mô hình
def prepare_data(data, size_window,size_predict,stepWindow):
    X, y = [], []
    startWindow = 0
    for i in range(len(data) - size_window - 1):
        if (len(data[(startWindow + size_window):(startWindow + size_window + size_predict) , 0]) != size_predict):
            break
        X.append(data[startWindow:(startWindow + size_window), :])
        y.append(data[(startWindow + size_window):(startWindow + size_window + size_predict) , 0])
          
       
        startWindow += stepWindow
    return np.array(X), np.array(y)


size_window = 7
size_predict = 1
stepWindow = 1
num_features=1
X_train, y_train = prepare_data(train_data.values, size_window, size_predict, stepWindow)
X_test, y_test = prepare_data(test_data.values, size_window, size_predict, stepWindow)

In [245]:
start_time = time.time()

model = Sequential()
model.add(Conv1D(filters=256, kernel_size=3, activation='relu', input_shape=(size_window, num_features)))
model.add(Conv1D(filters=256, kernel_size=3, activation='relu'))

model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mean_squared_error')

In [246]:
model.fit(X_train, y_train, epochs = 12000, batch_size = 64)


Epoch 1/12000
10/10 [==============================] - 1s 9ms/step - loss: 0.0513
Epoch 2/12000
10/10 [==============================] - 0s 9ms/step - loss: 0.0055
Epoch 3/12000
10/10 [==============================] - 0s 9ms/step - loss: 0.0029
Epoch 4/12000
10/10 [==============================] - 0s 7ms/step - loss: 0.0021
Epoch 5/12000
10/10 [==============================] - 0s 7ms/step - loss: 0.0016
Epoch 6/12000
10/10 [==============================] - 0s 7ms/step - loss: 0.0014
Epoch 7/12000
10/10 [==============================] - 0s 7ms/step - loss: 0.0014
Epoch 8/12000
10/10 [==============================] - 0s 7ms/step - loss: 0.0013
Epoch 9/12000
10/10 [==============================] - 0s 7ms/step - loss: 0.0012
Epoch 10/12000
10/10 [==============================] - 0s 7ms/step - loss: 0.0012
Epoch 11/12000
10/10 [==============================] - 0s 7ms/step - loss: 0.0011
Epoch 12/12000
10/10 [==============================] - 0s 7ms/step - loss: 0.0011
Epoch 13/1200

10/10 [==============================] - 0s 7ms/step - loss: 6.3233e-04
Epoch 189/12000
10/10 [==============================] - 0s 7ms/step - loss: 5.6966e-04
Epoch 190/12000
10/10 [==============================] - 0s 7ms/step - loss: 5.5741e-04
Epoch 191/12000
10/10 [==============================] - 0s 7ms/step - loss: 6.4427e-04
Epoch 192/12000
10/10 [==============================] - 0s 7ms/step - loss: 6.5265e-04
Epoch 193/12000
10/10 [==============================] - 0s 7ms/step - loss: 6.0120e-04
Epoch 194/12000
10/10 [==============================] - 0s 7ms/step - loss: 5.6047e-04
Epoch 195/12000
10/10 [==============================] - 0s 7ms/step - loss: 5.5121e-04
Epoch 196/12000
10/10 [==============================] - 0s 7ms/step - loss: 5.6396e-04
Epoch 197/12000
10/10 [==============================] - 0s 7ms/step - loss: 5.7740e-04
Epoch 198/12000
10/10 [==============================] - 0s 7ms/step - loss: 6.2244e-04
Epoch 199/12000
10/10 [=========================

10/10 [==============================] - 0s 7ms/step - loss: 5.3359e-04
Epoch 375/12000
10/10 [==============================] - 0s 7ms/step - loss: 4.8181e-04
Epoch 376/12000
10/10 [==============================] - 0s 7ms/step - loss: 5.2405e-04
Epoch 377/12000
10/10 [==============================] - 0s 7ms/step - loss: 4.8788e-04
Epoch 378/12000
10/10 [==============================] - 0s 7ms/step - loss: 5.2429e-04
Epoch 379/12000
10/10 [==============================] - 0s 7ms/step - loss: 5.0829e-04
Epoch 380/12000
10/10 [==============================] - 0s 7ms/step - loss: 5.3736e-04
Epoch 381/12000
10/10 [==============================] - 0s 7ms/step - loss: 5.5827e-04
Epoch 382/12000
10/10 [==============================] - 0s 7ms/step - loss: 5.2903e-04
Epoch 383/12000
10/10 [==============================] - 0s 7ms/step - loss: 5.5152e-04
Epoch 384/12000
10/10 [==============================] - 0s 7ms/step - loss: 5.3823e-04
Epoch 385/12000
10/10 [=========================

10/10 [==============================] - 0s 7ms/step - loss: 4.5788e-04
Epoch 561/12000
10/10 [==============================] - 0s 7ms/step - loss: 4.3836e-04
Epoch 562/12000
10/10 [==============================] - 0s 7ms/step - loss: 4.3529e-04
Epoch 563/12000
10/10 [==============================] - 0s 7ms/step - loss: 4.4666e-04
Epoch 564/12000
10/10 [==============================] - 0s 7ms/step - loss: 4.6064e-04
Epoch 565/12000
10/10 [==============================] - 0s 7ms/step - loss: 4.5089e-04
Epoch 566/12000
10/10 [==============================] - 0s 7ms/step - loss: 4.6313e-04
Epoch 567/12000
10/10 [==============================] - 0s 7ms/step - loss: 4.7254e-04
Epoch 568/12000
10/10 [==============================] - 0s 7ms/step - loss: 4.9780e-04
Epoch 569/12000
10/10 [==============================] - 0s 7ms/step - loss: 4.3596e-04
Epoch 570/12000
10/10 [==============================] - 0s 7ms/step - loss: 4.4151e-04
Epoch 571/12000
10/10 [=========================

10/10 [==============================] - 0s 7ms/step - loss: 3.9979e-04
Epoch 747/12000
10/10 [==============================] - 0s 7ms/step - loss: 4.0997e-04
Epoch 748/12000
10/10 [==============================] - 0s 7ms/step - loss: 3.9729e-04
Epoch 749/12000
10/10 [==============================] - 0s 7ms/step - loss: 3.9172e-04
Epoch 750/12000
10/10 [==============================] - 0s 7ms/step - loss: 4.3266e-04
Epoch 751/12000
10/10 [==============================] - 0s 7ms/step - loss: 4.3121e-04
Epoch 752/12000
10/10 [==============================] - 0s 7ms/step - loss: 5.1073e-04
Epoch 753/12000
10/10 [==============================] - 0s 7ms/step - loss: 4.6838e-04
Epoch 754/12000
10/10 [==============================] - 0s 7ms/step - loss: 4.3500e-04
Epoch 755/12000
10/10 [==============================] - 0s 7ms/step - loss: 4.4735e-04
Epoch 756/12000
10/10 [==============================] - 0s 7ms/step - loss: 4.1270e-04
Epoch 757/12000
10/10 [=========================

...

10/10 [==============================] - 0s 7ms/step - loss: 2.5931e-06
Epoch 11224/12000
10/10 [==============================] - 0s 7ms/step - loss: 2.1462e-06
Epoch 11225/12000
10/10 [==============================] - 0s 7ms/step - loss: 1.6392e-06
Epoch 11226/12000
10/10 [==============================] - 0s 7ms/step - loss: 1.3763e-06
Epoch 11227/12000
10/10 [==============================] - 0s 7ms/step - loss: 1.4267e-06
Epoch 11228/12000
10/10 [==============================] - 0s 7ms/step - loss: 1.2810e-06
Epoch 11229/12000
10/10 [==============================] - 0s 7ms/step - loss: 8.6879e-07
Epoch 11230/12000
10/10 [==============================] - 0s 7ms/step - loss: 8.9014e-07
Epoch 11231/12000
10/10 [==============================] - 0s 7ms/step - loss: 8.5954e-07
Epoch 11232/12000
10/10 [==============================] - 0s 7ms/step - loss: 9.4385e-07
Epoch 11233/12000
10/10 [==============================] - 0s 7ms/step - loss: 1.0043e-06
Epoch 11234/12000
10/10 [===

10/10 [==============================] - 0s 7ms/step - loss: 1.0006e-05
Epoch 11406/12000
10/10 [==============================] - 0s 8ms/step - loss: 1.4645e-05
Epoch 11407/12000
10/10 [==============================] - 0s 8ms/step - loss: 2.5074e-05
Epoch 11408/12000
10/10 [==============================] - 0s 7ms/step - loss: 2.4397e-05
Epoch 11409/12000
10/10 [==============================] - 0s 7ms/step - loss: 2.5927e-05
Epoch 11410/12000
10/10 [==============================] - 0s 7ms/step - loss: 2.6558e-05
Epoch 11411/12000
10/10 [==============================] - 0s 7ms/step - loss: 2.7823e-05
Epoch 11412/12000
10/10 [==============================] - 0s 7ms/step - loss: 2.0284e-05
Epoch 11413/12000
10/10 [==============================] - 0s 7ms/step - loss: 1.6346e-05
Epoch 11414/12000
10/10 [==============================] - 0s 7ms/step - loss: 2.2542e-05
Epoch 11415/12000
10/10 [==============================] - 0s 8ms/step - loss: 1.2928e-05
Epoch 11416/12000
10/10 [===

10/10 [==============================] - 0s 8ms/step - loss: 2.1279e-06
Epoch 11588/12000
10/10 [==============================] - 0s 7ms/step - loss: 1.8112e-06
Epoch 11589/12000
10/10 [==============================] - 0s 7ms/step - loss: 3.5712e-06
Epoch 11590/12000
10/10 [==============================] - 0s 7ms/step - loss: 3.8600e-06
Epoch 11591/12000
10/10 [==============================] - 0s 7ms/step - loss: 3.6324e-06
Epoch 11592/12000
10/10 [==============================] - 0s 7ms/step - loss: 4.2419e-06
Epoch 11593/12000
10/10 [==============================] - 0s 7ms/step - loss: 5.0904e-06
Epoch 11594/12000
10/10 [==============================] - 0s 7ms/step - loss: 5.9802e-06
Epoch 11595/12000
10/10 [==============================] - 0s 7ms/step - loss: 8.6129e-06
Epoch 11596/12000
10/10 [==============================] - 0s 7ms/step - loss: 8.2008e-06
Epoch 11597/12000
10/10 [==============================] - 0s 7ms/step - loss: 7.5937e-06
Epoch 11598/12000
10/10 [===

10/10 [==============================] - 0s 8ms/step - loss: 1.7012e-06
Epoch 11770/12000
10/10 [==============================] - 0s 7ms/step - loss: 1.2184e-06
Epoch 11771/12000
10/10 [==============================] - 0s 7ms/step - loss: 1.3453e-06
Epoch 11772/12000
10/10 [==============================] - 0s 7ms/step - loss: 9.0182e-07
Epoch 11773/12000
10/10 [==============================] - 0s 7ms/step - loss: 1.0078e-06
Epoch 11774/12000
10/10 [==============================] - 0s 7ms/step - loss: 1.0624e-06
Epoch 11775/12000
10/10 [==============================] - 0s 7ms/step - loss: 1.3956e-06
Epoch 11776/12000
10/10 [==============================] - 0s 7ms/step - loss: 1.5424e-06
Epoch 11777/12000
10/10 [==============================] - 0s 7ms/step - loss: 2.0901e-06
Epoch 11778/12000
10/10 [==============================] - 0s 7ms/step - loss: 1.5750e-06
Epoch 11779/12000
10/10 [==============================] - 0s 7ms/step - loss: 1.7105e-06
Epoch 11780/12000
10/10 [===

10/10 [==============================] - 0s 7ms/step - loss: 8.7662e-06
Epoch 11952/12000
10/10 [==============================] - 0s 7ms/step - loss: 4.9788e-06
Epoch 11953/12000
10/10 [==============================] - 0s 7ms/step - loss: 3.9929e-06
Epoch 11954/12000
10/10 [==============================] - 0s 7ms/step - loss: 3.3736e-06
Epoch 11955/12000
10/10 [==============================] - 0s 7ms/step - loss: 3.6299e-06
Epoch 11956/12000
10/10 [==============================] - 0s 7ms/step - loss: 2.9401e-06
Epoch 11957/12000
10/10 [==============================] - 0s 7ms/step - loss: 2.8053e-06
Epoch 11958/12000
10/10 [==============================] - 0s 7ms/step - loss: 3.4032e-06
Epoch 11959/12000
10/10 [==============================] - 0s 7ms/step - loss: 4.5410e-06
Epoch 11960/12000
10/10 [==============================] - 0s 7ms/step - loss: 3.3909e-06
Epoch 11961/12000
10/10 [==============================] - 0s 7ms/step - loss: 3.5078e-06
Epoch 11962/12000
10/10 [===

In [247]:
y_pred = model.predict(X_test)
end_time = time.time()
training_time = end_time - start_time
print(f"Training time: {training_time} seconds")



5/5 [==============================] - 0s 2ms/step
Training time: 856.445464849472 seconds


In [248]:
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

mape = mean_absolute_percentage_error(y_test, y_pred)
# In giá trị MSE
print("MSE: ", mse)
print('RMSE: ', rmse)
print('MAPE : ', mape)

MSE:  0.016621228840705507
RMSE:  0.12892334482437812
MAPE :  0.135719843734157
